Hi, Marcel! 

Hope it'll help you to run my code quicker :)

Alex

## Input format

| ID |     Name    | FA_Name | Value |
|---:|:-----------:|:-------:|-------|
|  1 | Revenue_123 | Revenue | -100  |
where **ID** - is actual Business Processes ID, **Name** is the full name of FinancialAccount (not sure that it's required, but initially as I guess one has something like "Revenue_1", "Revenue_2" for different product and only after application of Coarse-graining financial accounts we are getting **FA_Name** value. **Value** is just a amount.
## Extraction signatures of BP
For that purpose I've implemented aggregation function over *groupby()* object, as input it uses one BP – a part of original DF with the same ID value – and return a string like
```
'([(\\'Revenue\\', 0.8336558167088283), (\\'Tax\\', 0.1663441832911717)], [(\\'TradeReceivables\\', 1.0)])'
```
then we leave only unique signatures (SAME FA from left and right part AND EXACTLY EQUAL coefficients).
## Data preprocessing
Then on that DataFrame you need to apply [*prepare_data()*](https://github.com/AlexWorldD/NetEmbs/blob/master/NetEmbs/DataProcessing/prepare_data.py) - please read the documentation to that function. The last thing I've added is 
```
 :param unique: True if Data has to be filtered wrt to Signatures of BPs
 ```

After that you should get the following structure:

| ID |     Name    | FA_Name | Value | Debit | Credit | from |
|---:|:-----------:|:-------:|-------|-------|--------|------|
|  1 | Revenue_123 | Revenue | -100  | 0.0   | 1.0    | True |

here **from** is for visualization purposes only (different colors of ingoing/outgoing edges)
## Construct FSN
Now we can construct FSN from our DataFrame, *build()* method of *FSN* class. 

I recommend to check the construction via *get_FA()* method, it should return the set of Financial Account nodes.
## RandomWalks
#### Already implemented step strategies
1. Pure RandomWalk (uniform probabilities, follows the direction)
2. MetaDef (change directions, uniform probabilities)
3. MetaDefWeighted (change directions, weighted probabilities "rich gets richer")
4. MetaDiff (change directions, probabilities depend on the differences between edges)

For test purpose you can try *randomWalk()* from NetEmbs.FSN.utils, it returns the sampled sequence. 

**BUT**, I've implemented special helper funciton for adding to original DataFrame a column with N top joint-appearance nodes. It's *add_similar()* function from NetEmbs.FSN.utils.

You also can test it with different step strategies: "DefUniform", "DefWeighted", "MetaUniform", "MetaWeighted", "MetaDiff" (version argument).

### Example with sampled data

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import networkx as nx
from networkx.algorithms import bipartite
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
plt.rcParams["figure.figsize"] = [20,10]
from NetEmbs.DataProcessing.normalize import normalize
from NetEmbs.DataProcessing.connect_db import *
from NetEmbs.GenerateData.complex_df import sales_collections
d = upload_data("Simulation/FSN_Data.db", limit=40)
d.head(5)

,ID,Name,FA_Name,Value
0,1,Revenue_2,Revenue,-668.000000
1,1,Tax_2,Tax,-140.497744
2,1,TradeReceivables_2,TradeReceivables,808.497744
3,2,CostOfSales_11,CostOfSales,668.000000
4,2,Inventory_11,Inventory,-668.000000


In [30]:
from NetEmbs.DataProcessing.rename_columns import rename_columns
from NetEmbs.GenerateData.complex_df import sales_collections
from NetEmbs.DataProcessing.prepare_data import prepare_data
from NetEmbs.DataProcessing.unique_signatures import unique_BPs
from NetEmbs.DataProcessing.splitting import add_from_column
test_df = pd.DataFrame({'transactionID': {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 2, 6: 2, 7: 2},
 'accountID': {0: 'Revenue',
  1: 'Tax',
  2: 'Trade Receivables',
  3: 'Trade Receivables',
  4: 'Cash',
  5: 'Revenue',
  6: 'Tax',
  7: 'Trade Receivables'},
 'Journal': {0: 'Sales ledger',
  1: 'Sales ledger',
  2: 'Sales ledger',
  3: 'Sales ledger',
  4: 'Sales ledger',
  5: 'Sales ledger',
  6: 'Sales ledger',
  7: 'Sales ledger'},
 'Date': {0: '01/01/2017',
  1: '01/01/2017',
  2: '01/01/2017',
  3: '01/01/2017',
  4: '01/01/2017',
  5: '01/01/2017',
  6: '01/01/2017',
  7: '01/01/2017'},
 'debitAmount': {0: 0.0,
  1: 0.0,
  2: 429.18,
  3: 0.0,
  4: 429.18,
  5: 0.0,
  6: 0.0,
  7: 1059.82},
 'creditAmount': {0: 403.0,
  1: 26.18,
  2: 0.0,
  3: 429.18,
  4: 0.0,
  5: 997.0,
  6: 62.82,
  7: 0.0}})
# test_df = unique_BPs(rename_columns(test_df))
# for group in test_df.groupby("ID"):
#     print(group[1]["Debit"])
test_df = add_from_column(rename_columns(test_df))
max(test_df['from'].values ==(test_df['Credit']>0).values)

True

In [ ]:
from NetEmbs.DataProcessing import split_to_debit_credit
split_to_debit_credit(d)

In [3]:
from NetEmbs.DataProcessing import *
d = prepare_data(d)
d.head(10)

,ID,Signature,Name,FA_Name,Value,Debit,Credit,from
0,1,"([('Revenue', 0.8262237029131959), ('Tax', 0.1...",Revenue_2,Revenue,-668.000000,0.000000,0.826224,True
1,1,"([('Revenue', 0.8262237029131959), ('Tax', 0.1...",Tax_2,Tax,-140.497744,0.000000,0.173776,True
2,1,"([('Revenue', 0.8262237029131959), ('Tax', 0.1...",TradeReceivables_2,TradeReceivables,808.497744,1.000000,0.000000,False
3,2,"([('Inventory', 1.0)], [('CostOfSales', 1.0)])",CostOfSales_11,CostOfSales,668.000000,1.000000,0.000000,False
4,2,"([('Inventory', 1.0)], [('CostOfSales', 1.0)])",Inventory_11,Inventory,-668.000000,0.000000,1.000000,True
5,3,"([('Revenue', 0.8224210569468975), ('Tax', 0.1...",Revenue_3,Revenue,-190.000000,0.000000,0.822421,True
6,3,"([('Revenue', 0.8224210569468975), ('Tax', 0.1...",Tax_3,Tax,-39.602122,0.000000,0.171419,True
7,3,"([('Revenue', 0.8224210569468975), ('Tax', 0.1...",mbfehy_3,mbfehy,-1.423091,0.000000,0.006160,True
8,3,"([('Revenue', 0.8224210569468975), ('Tax', 0.1...",TradeReceivables_3,TradeReceivables,231.025213,1.000000,0.000000,False
9,4,"([('oklfmo', 0.005831805480474793), ('Inventor...",qpfxva_8,qpfxva,0.273129,0.001438,0.000000,False


In [ ]:
from NetEmbs.FSN.graph import FSN
fsn = FSN()
fsn.build(d, name_column="FA_Name")
fsn.nodes()

In [ ]:
fsn.get_BP()

In [ ]:
from NetEmbs.Vis.plots import plotFSN
plotFSN(fsn, edge_labels=False, node_labels=False)

In [ ]:
from NetEmbs.FSN.utils import add_similar, get_pairs
add_similar(d)

In [ ]:
ID NAME Cred, Deb
1 Revenue   121 Tax.  21 TR 100
ID NAME Cred, Deb
2 Revenue   1210 Tax.  210 TR 1000
---
Normalize, then
ID NAME Cred, Deb
1 Revenue   0.8 Tax.  0.2 TR 1.0